In [1]:
from functions import *
import glob
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [2]:
IMAGE_DIR = "./data/data/data"

def load():
    file_list = glob.glob(IMAGE_DIR + "/*.jpg")
    X = []
    Y = []

    for fname in file_list:
        with Image.open(fname) as img:
            np_img = np.array(img).flatten()
        label = int(os.path.split(fname)[-1].split('.')[0].split('_')[3])-1

        X.append(np_img)
        tempy = np.zeros(15)
        tempy[label] = 1
        Y.append(tempy)
    X, Y = np.array(X), np.array(Y)
    return(X, Y)

In [3]:
# create test, train split
X, Y = load()
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.8, random_state=2021)
Ytrain_classes = []
for i in range(Ytrain.shape[1]):
    Ytrain_0 = classify(Ytrain, i)
    Ytrain_classes.append(Ytrain_0)
    print(f"{Ytrain_0.count(1)} datapoints have classification {i}")

791 datapoints have classification 0
806 datapoints have classification 1
812 datapoints have classification 2
806 datapoints have classification 3
798 datapoints have classification 4
781 datapoints have classification 5
805 datapoints have classification 6
832 datapoints have classification 7
814 datapoints have classification 8
785 datapoints have classification 9
792 datapoints have classification 10
799 datapoints have classification 11
781 datapoints have classification 12
800 datapoints have classification 13
798 datapoints have classification 14


In [4]:
def update_weights(weights, best):
    for i in range(len(weights)):
        if i not in best.miss_data:
            weights[i] = 1/2*(1/(1 - best.error_rate))*weights[i]
        else:
            weights[i] = 1/2*(1/best.error_rate)*weights[i]

def run(Weak_Learners, data, eval_set, H, weights):
    for wl in Weak_Learners:
        wl.miss_classify(data, eval_set)
        wl.calc_error_rate(weights)
    
    best = Weak_Learners[0]
    for wl in Weak_Learners:
        if wl.error_rate < best.error_rate:
            best = wl
    
    best.calc_voting_power()
    H.append(best)
    
    update_weights(weights, best)
    
    accuracy = H_accuracy(H, data, eval_set)
    
    return accuracy

In [ ]:
"""
Here we test the performace of our model with different amounts of training data 
TODO: METHOD OF CALCULATING ACCURACY MAY BE INCORRECT
"""
trainSizes = [x / 10 for x in range(1, 10, 1)] # Create a list of [0.1, 0.2, 0.3, ..., 0.9]

for trainRatio in trainSizes:
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=trainRatio, random_state=2021)
    Ytrain_classes = []
    Ytest_classes = []
    for i in range(Ytrain.shape[1]):
        Ytrain_i = classify(Ytrain, i)
        Ytrain_classes.append(Ytrain_i)
        Ytest_i = classify(Ytest, i)
        Ytest_classes.append(Ytest_i)
    
    accuracies = []
    for i in range(Ytrain.shape[1]):
        H = []
        Weak_Learners = []
        for _ in range(10):
            model = ShallowTree()
            model.fit(Xtrain, Ytrain_classes[0])
            Weak_Learners.append(WeakLearner(model, 0))

        weights = np.array([1/len(Xtrain) for _ in range(len(Xtrain))])
        local_accuracies = []
        for r in range(10):
            local_accuracies.append(run(Weak_Learners, Xtrain, Ytrain_classes[i], H, weights))
        accuracies.append(sum(local_accuracies)/len(local_accuracies))
        training_accuracy = H_accuracy(H, Xtrain, Ytrain_classes[i])
        print(f"Training accuracy for class {i}: {training_accuracy:.2%}")
    #     WL.miss_classify(Xtrain, Ytrain_classes[0])
    #     WL.calc_error_rate(np.array([1 for d in range(Xtrain.shape[0])]))
    print(f"Ratio: {trainRatio:.2%}\n\tAccuracy: {sum(accuracies)/len(accuracies):.2%}")


Training accuracy for class 0: 90.93%
Training accuracy for class 1: 82.13%
Training accuracy for class 2: 82.53%
Training accuracy for class 3: 82.13%
Training accuracy for class 4: 80.53%
Training accuracy for class 5: 80.93%
Training accuracy for class 6: 81.33%
Training accuracy for class 7: 80.00%
Training accuracy for class 8: 81.20%
Training accuracy for class 9: 82.40%
Training accuracy for class 10: 82.93%
Training accuracy for class 11: 82.00%
Training accuracy for class 12: 83.33%
Training accuracy for class 13: 80.27%


In [93]:
""" 
Now we vary the size of the trees
"""
# I had to do this because of import issues
# TODO: Fix this so you don't have to redefine the function here
def ShallowTree(d = 2): 
    return DecisionTreeClassifier(max_depth=d)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.8, random_state=2021)
Ytrain_classes = []
for i in range(Ytrain.shape[1]):
    Ytrain_0 = classify(Ytrain, i)
    Ytrain_classes.append(Ytrain_0)

for treeSize in range(2, 10):
    Ytrain_classes = []
    for i in range(Ytrain.shape[1]):
        Ytrain_0 = classify(Ytrain, i)
        Ytrain_classes.append(Ytrain_0)

    model = ShallowTree(treeSize)
    model.fit(Xtrain, Ytrain_classes[0])
    WL = WeakLearner(model, 0)
    WL.miss_classify(Xtrain, Ytrain_classes[0])
    WL.calc_error_rate(np.array([1 for d in range(Xtrain.shape[0])]))
    print("Percent correct with tree size {}: {}%".format(treeSize, (WL.error_rate / Xtest.shape[0]) * 100))

Percent correct with tree size 2: 22.633333333333333%
Percent correct with tree size 3: 20.200000000000003%
Percent correct with tree size 4: 16.066666666666666%
Percent correct with tree size 5: 13.433333333333334%
Percent correct with tree size 6: 10.233333333333333%
Percent correct with tree size 7: 7.433333333333334%
Percent correct with tree size 8: 6.033333333333333%
Percent correct with tree size 9: 5.066666666666666%
